<a href="https://colab.research.google.com/github/AnanyaSourav/Ayurvedic-app-NIPGR/blob/main/admet%3Eeffectiveness_percentage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import pandas as pd

# Step 1. Define your SMILES list
smiles_list = [
    "CCOc1ccc2nc(S(N)(=O)=O)sc2c1",   # Example: sulfanilamide
    "C1=CC=CC=C1",                    # Benzene
    "CC(=O)OC1=CC=CC=C1C(=O)O"        # Aspirin (acetylsalicylic acid)
    # Add more SMILES as needed
]

# Step 2. Function to batch-query ADMETlab 3.0
def get_admet_admetlab(smiles_list):
    results = []
    url = "https://admetlab3.scbdd.com/api/predict/"
    headers = {'Content-Type': 'application/json'}
    for sm in smiles_list:
        payload = {"structure": sm}
        try:
            r = requests.post(url, json=payload, headers=headers)
            if r.status_code == 200:
                out = r.json()
                props = out.get("properties", {})
                result = {
                    "SMILES": sm,
                    "HIA": props.get("HumanIntestinalAbsorption", None),
                    "BBB": props.get("BloodBrainBarrier", None),
                    "Ames_test": props.get("AmesTest", None),
                    "hERG": props.get("HERGInhibition", None),
                    # Add more endpoints as desired
                }
                results.append(result)
            else:
                results.append({"SMILES": sm, "HIA": None, "BBB": None, "Ames_test": None, "hERG": None})
            time.sleep(0.5)  # be kind to the API
        except Exception as e:
            results.append({"SMILES": sm, "Error": str(e)})
    return pd.DataFrame(results)

# Step 3. Effectiveness scoring function
def calculate_efficiency(df):
    # Apply simple binary rules; modify as required for your needs.
    df["score_HIA"] = df["HIA"].apply(lambda x: 1 if str(x).lower() in ["high", "positive", "yes"] else 0)
    df["score_BBB"] = df["BBB"].apply(lambda x: 1 if str(x).lower() in ["no", "negative"] else 0)
    df["score_Ames"] = df["Ames_test"].apply(lambda x: 1 if "non" in str(x).lower() else 0)
    df["score_hERG"] = df["hERG"].apply(lambda x: 1 if "non" in str(x).lower() else 0)
    score_cols = ["score_HIA", "score_BBB", "score_Ames", "score_hERG"]
    df["ADMET_Efficiency_%"] = df[score_cols].mean(axis=1) * 100
    return df

# Step 4. Run the pipeline
admet_df = get_admet_admetlab(smiles_list)
final_df = calculate_efficiency(admet_df)

# Step 5. Calculate average effectiveness across all compounds
average_efficiency = final_df["ADMET_Efficiency_%"].mean()

# Display results
print("Individual ADMET effectiveness (%):")
print(final_df[["SMILES", "ADMET_Efficiency_%"]])
print(f"\nAverage ADMET effectiveness: {average_efficiency:.2f}%")
